In [4]:
import pandas as pd

In [5]:
data = r"clean_cad.csv"
clean_data = pd.read_csv(data)
clean_data["Call Time"] = pd.to_datetime(clean_data["Call Time"])
clean_data

,Unnamed: 0,Call Time,Reason for Dispatch,Call Sign,2 Call Sign,Disposition
0,0,2016-01-01 00:00:04,ASSAULT,5E57,5E57,ADVISED
1,1,2016-01-01 00:00:04,ASSAULT,5E57,4X40,ADVISED
2,2,2016-01-01 00:00:04,ASSAULT,5E57,4E53,ADVISED
3,3,2016-01-01 00:00:24,TRAFFIC STOP,5T81,5T81,ADVISED
4,4,2016-01-01 00:02:45,CHECK WELFARE,3X90,3F61,ASSISTED
...,...,...,...,...,...,...
1267985,1616812,2023-12-31 22:44:38,DISPUTE,6E19,4K97,REPORT TAKEN
1267986,1616813,2023-12-31 22:44:38,DISPUTE,6E19,6E19,REPORT TAKEN
1267987,1616814,2023-12-31 22:44:38,DISPUTE,6E19,5X51,REPORT TAKEN
1267988,1616817,2023-12-31 22:56:40,TRAFFIC STOP,4F74,4F72,UNIFORM TRAFFIC CITATION ISSUED


In [6]:
target_call_signs = ["1J77", "3J78", "4J79"]
cahoots_call_signs = target_call_signs

In [7]:
#Manually Finding the first occurrence of each call sign

filtered_call_signs = clean_data[clean_data['Call Sign'].isin(target_call_signs)]

first_occurrence_call_sign = filtered_call_signs.groupby('Call Sign')['Call Time'].min()
first_1J77 = first_occurrence_call_sign['1J77']
first_3J78 = first_occurrence_call_sign['3J78']
first_4J79 = first_occurrence_call_sign['4J79']

last_occurrence = filtered_call_signs.groupby('Call Sign')['Call Time'].max()
last_1J77 = last_occurrence['1J77']
last_3J78 = last_occurrence['3J78']
last_4J79 = pd.to_datetime('2022-03-01')


times_1J77 = {
    "First": first_1J77,
    "Last": last_1J77
}
times_3J78 = {
    "First": first_3J78,
    "Last": last_3J78
}
times_4J79 = {
    "First": first_4J79,
    "Last": last_4J79
}
call_sign_times = {
    "3J78": times_3J78,
    "1J77": times_1J77,
    "4J79": times_4J79
}

In [9]:
def calculate_counts(table: pd.DataFrame, call_signs: list):
    table['Call Time'] = pd.to_datetime(table['Call Time'])

    monthly_call_count = table['Call Time'].dt.to_period('M').value_counts().rename('Total Calls')

    monthly_cahoots = table[table['Call Sign'].isin(call_signs)]
    monthly_call_sign = monthly_cahoots['Call Time'].dt.to_period('M').value_counts().rename('Cahoots Calls')

    t_v_c = pd.DataFrame({'Total Calls': monthly_call_count, 'Cahoots Calls': monthly_call_sign}).fillna(0)
    t_v_c.index = t_v_c.index.to_timestamp()
    t_v_c.reset_index(inplace=True)

    t_v_c['Percentage'] = round(t_v_c['Cahoots Calls'] / t_v_c['Total Calls'], 4) * 100
    return t_v_c